In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

!pip3 install datasets
import datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
dataset = datasets.load_dataset("ccdv/arxiv-summarization", split = "train", streaming = True) #acts like a generator; takes data as you call for it
raw_dataset = list(dataset.take(3500))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

In [ ]:
# BATCH SIZE: 4 (papers)
# CHUNK SIZE: 5 (each paper broken into 5 chunks of n tokens each)




#        forward pass 1 | FP 2    | FP 3    | FP 4    | FP 5    |
#
# paper 1:      chunk 1 | chunk 2 | chunk 3 | chunk 4 | chunk 5 |
# paper 2:      chunk 1 | chunk 2 | chunk 3 | chunk 4 | chunk 5 |
# paper 3:      chunk 1 | chunk 2 | chunk 3 | chunk 4 | chunk 5 |
# paper 4:      chunk 1 | chunk 2 | chunk 3 | chunk 4 | chunk 5 |
#
#
#
#        forward pass 6 | FP 7    | FP 8    | FP 9    | FP 10   |
#
# paper 5:      chunk 1 | chunk 2 | chunk 3 | chunk 4 | chunk 5 |
# paper 6:      chunk 1 | chunk 2 | chunk 3 | chunk 4 | chunk 5 |
# paper 7:      chunk 1 | chunk 2 | chunk 3 | chunk 4 | chunk 5 |
# paper 8:      chunk 1 | chunk 2 | chunk 3 | chunk 4 | chunk 5 |


In [ ]:
segments = 10
segment_length = 512 #feed 512 tokens to the model
chunk_size = segments * segment_length
chunk_size

5120

In [ ]:
raw_articles = [x['article'] for x in raw_dataset]
raw_articles = [x for x in raw_articles if len(x) > 5120]

In [ ]:
print ("number of articles", len(raw_articles))

unique_chars = set(''.join([i for i in raw_articles]))
print("character set length", len(unique_chars))
print("character set", ''.join(sorted(unique_chars)))

number of articles 3401
character set length 70
character set 
 !"#$%&'()*+,-./0123456789:;<=>?@[\]^_`abcdefghijklmnopqrstuvwxyz{|}~


In [ ]:
np.fromstring(raw_articles[0], dtype = np.uint8)[:512] #character set being mapped to 128 integers

<ipython-input-32-292acdd9a5c1>:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  np.fromstring(raw_articles[0], dtype = np.uint8)[:512] #character set being mapped to 128 integers


array([ 97, 100, 100, 105, 116, 105, 118, 101,  32, 109, 111, 100, 101,
       108, 115,  32,  64, 120,  99, 105, 116, 101,  32, 112, 114, 111,
       118, 105, 100, 101,  32,  97, 110,  32, 105, 109, 112, 111, 114,
       116,  97, 110, 116,  32, 102,  97, 109, 105, 108, 121,  32, 111,
       102,  32, 109, 111, 100, 101, 108, 115,  32, 102, 111, 114,  32,
       115, 101, 109, 105, 112,  97, 114,  97, 109, 101, 116, 114, 105,
        99,  32, 114, 101, 103, 114, 101, 115, 115, 105, 111, 110,  32,
       111, 114,  32,  99, 108,  97, 115, 115, 105, 102, 105,  99,  97,
       116, 105, 111, 110,  32,  46,  32, 115, 111, 109, 101,  32, 114,
       101,  97, 115, 111, 110, 115,  32, 102, 111, 114,  32, 116, 104,
       101,  32, 115, 117,  99,  99, 101, 115, 115,  32, 111, 102,  32,
        97, 100, 100, 105, 116, 105, 118, 101,  32, 109, 111, 100, 101,
       108, 115,  32,  97, 114, 101,  32, 116, 104, 101, 105, 114,  32,
       105, 110,  99, 114, 101,  97, 115, 101, 100,  32, 102, 10

In [ ]:
def decode_text(tokens):
  return ''.join([chr(i) for i in tokens])


decode_text(np.fromstring(raw_articles[0], dtype = np.uint8)[:512])

<ipython-input-33-507a6a0e17f7>:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  decode_text(np.fromstring(raw_articles[0], dtype = np.uint8)[:512])


'additive models @xcite provide an important family of models for semiparametric regression or classification . some reasons for the success of additive models are their increased flexibility when compared to linear or generalized linear models and their increased interpretability when compared to fully nonparametric models . \n it is well - known that good estimators in additive models are in general less prone to the curse of high dimensionality than good estimators in fully nonparametric models . \n many ex'

In [ ]:
raw_articles[0][:512]

'additive models @xcite provide an important family of models for semiparametric regression or classification . some reasons for the success of additive models are their increased flexibility when compared to linear or generalized linear models and their increased interpretability when compared to fully nonparametric models . \n it is well - known that good estimators in additive models are in general less prone to the curse of high dimensionality than good estimators in fully nonparametric models . \n many ex'

In [ ]:
converted = [np.fromstring(doc, dtype=np.int8) for doc in raw_articles] #tokenized format

<ipython-input-35-f37bb218e19b>:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  converted = [np.fromstring(doc, dtype=np.int8) for doc in raw_articles] #tokenized format


In [ ]:
def clip_article(doc, chunk_size):
    remainder = len(doc) % chunk_size
    return doc[:-remainder] #removes last remainder tokens

clipped = [clip_article(doc, 5120) for doc in converted]

In [ ]:
clipped[1].shape[0] / 5120

3.0

In [ ]:
chunked = [doc.reshape(-1, chunk_size) for doc in clipped] #chunks of clipped

In [ ]:
processed_data = torch.tensor(np.concatenate(chunked), dtype=torch.long)
processed_data.shape

torch.Size([20853, 5120])

In [ ]:
eighty_split = int(processed_data.shape[0] * .8)
ninety_split = int(processed_data.shape[0] * .9)

In [ ]:
train_loader = iter(DataLoader(processed_data[:eighty_split], batch_size = 8, shuffle = True))
val_loader = iter(DataLoader(processed_data[eighty_split:ninety_split], batch_size = 8, shuffle = True))
test_loader = iter(DataLoader(processed_data[ninety_split:], batch_size = 8, shuffle = True))

In [ ]:
loader = iter(DataLoader(processed_data, batch_size=8, shuffle=True))

In [ ]:
example = next(loader)

In [ ]:
example.shape

torch.Size([8, 5120])

In [ ]:
seq, labels = example[:, :-1], example[:, 1:]

In [ ]:
seq[0][:15]

tensor([103, 114,  97, 118, 105, 116,  97, 116, 105, 111, 110,  97, 108,  32,
        108])

In [ ]:
labels[0][:15] #basically a shift by 1 (predicting next character)

tensor([114,  97, 118, 105, 116,  97, 116, 105, 111, 110,  97, 108,  32, 108,
        101])

In [ ]:
labels.shape

torch.Size([8, 5119])

In [ ]:
seq.chunk(10, dim=1)[0].shape #10 different traning sequences

torch.Size([8, 512])

In [ ]:
for seq_segment, labels_segment in zip(seq.chunk(10, dim = -1), labels.chunk(10, dim = -1)):
    print(decode_text(seq_segment[1]), "\n *********** \n") #continuation of text

me masa @xmath0 in @xmath2 such that @xmath39    choose a masa @xmath141 such that @xmath9 belongs to @xmath141 . 
 theorem ( [ shm1 ] ) yields that there is a unitary @xmath22 in @xmath68 and a projection @xmath43 in @xmath141 such that @xmath388 choose a masa @xmath389 in @xmath281 that contains @xmath289 . 
 then , we have that @xmath390    note that we have that @xmath391 . 
 let @xmath0 be the masa @xmath392 . 
 we then get by applying the automorphism @xmath393 to the equation ( [ fc ] ) that @xmath39 
 *********** 

4 since @xmath395 , we are done . 
 the above theorem , as remarked in the introduction , is a natural generalization of the schur - horn theorem to type @xmath1 factors . 
 this generalization does not directly imply the alternative conjecture of arveson and kadison from @xcite . 
 we prove the conjecture in full in the next section . 
 we now turn to the second natural generalization of the schur - horn theorem . 
 the theorem of the last section characterizes the 

In [ ]:
model = nn.Sequential(
    nn.Embedding(128,16), # (vocab_size, embedding_dim)
    nn.Linear(16, 150),
    nn.ReLU(),
    nn.Linear(150,150),
    nn.ReLU(),
    nn.Linear(150, 128), # (params, vocab_size)
)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
model.train()

segments = 10

for i in range(200):

    data = next(train_loader) # (batch_size, sequence_length) # (8, 5120)
    seq, labels = data[:, :-1], data[:, 1:]
    train_loss = 0.
    model.train()

    for seq_segment, labels_segment in zip(seq.chunk(segments, dim = -1), labels.chunk(segments, dim = -1)): # ten passes of (8, 512)
        optimizer.zero_grad()
        y_pred = model(seq_segment)
        y_pred = y_pred.transpose(2,1)
        loss = loss_fn(y_pred, labels_segment)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    if i % 5 == 0:
        print (train_loss / segments)

    if i > 0 and i % 50 == 0:
        val_data = next(val_loader)
        seq, labels = val_data[:, :-1], val_data[:, 1:]
        eval_loss = 0.
        model.eval()
        for seq_segment, labels_segment in zip(seq.chunk(segments, dim = -1), labels.chunk(segments, dim = -1)): # ten passes of (8, 512)
            with torch.no_grad():
                y_pred = model(seq_segment)
                y_pred = y_pred.transpose(2,1)
                loss = loss_fn(y_pred, labels_segment)
                eval_loss += loss.item()

        print ("VALIDATION LOSS", (eval_loss / segments))


4.76202802658081
3.4509190559387206
3.049196982383728
3.031331992149353
2.8706095457077025
2.7924951314926147
2.8087601184844972
2.7336995601654053
2.6220270872116087
2.671736168861389
2.622007131576538
VALIDATION LOSS 2.842620325088501
2.6414557456970216
2.621060037612915
2.5965858936309814
2.6283721685409547
2.538051629066467
2.5659662961959837
2.5906848907470703
2.6154890298843383
2.5480950593948366
2.4774542808532716
VALIDATION LOSS 2.8872291326522825
2.4991343021392822
2.534153127670288
2.4870404958724976
2.802467131614685
2.418036699295044
2.46250524520874
2.4495493888854982
2.4671412229537966
2.608990263938904
2.4182801246643066
VALIDATION LOSS 2.426920199394226
2.490543818473816
2.461560344696045


KeyboardInterrupt: 

In [ ]:
test_data = next(test_loader)
seq, labels = test_data[:, :-1], test_data[:, 1:]
test_loss = 0.
model.eval()
for seq_segment, labels_segment in zip(seq.chunk(segments, dim = -1), labels.chunk(segments, dim = -1)): # ten passes of (8, 512)
    with torch.no_grad():
        y_pred = model(seq_segment)
        y_pred = y_pred.transpose(2,1)
        loss = loss_fn(y_pred, labels_segment)
        test_loss += loss.item()

print ("TEST LOSS", (test_loss / segments))

TEST LOSS 2.4852545738220213
